In [32]:
import pandas as pd
import regex as re
import numpy as np
import copy
import spacy
from spacy.tokens import Doc
from spacy.attrs import LOWER, POS, ENT_TYPE, IS_ALPHA
from spacy.matcher import Matcher
from spacy.util import filter_spans
import re
import matplotlib.pyplot as plt
from collections import Counter
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import pyplot as plt


from DocumentWorking import Document
import sys, importlib

nlp = spacy.load("en_use_md")

/home/blitt/.local/lib/python3.8/site-packages/spacy/util.py:837: UserWarning: [W095] Model 'en_use_md' (0.4.3) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.3.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [33]:
def getSpans(spaCyDoc): 
    #now we want to get spans on punctuation 
    spanList = []
    spanStart = 0 
    spanEnd = 0
    for token in spaCyDoc: 
        if token.is_punct: 
            currSpan = spaCyDoc[spanStart:spanEnd]
            spanList.append(currSpan)
            spanStart = spanEnd + 1
        spanEnd += 1
    return spanList

#this inList will be a list of spans 
#TODO: make function that adds noun vectors together and uses those. Also consider 
#how distance should be defined here 
def extractNouns(inList): 
    outList = []
    #get only nouns from the spans 
    for item in inList: 
        tokenList = []
        for token in item: 
            if token.pos_ == "NOUN": 
                tokenList.append(token)
                print(token.vector)
        outList.append(tokenList)
    return outList

In [15]:
#load in Document and nlp modules/objects
from DocumentWorking import Document
import sys, importlib
importlib.reload(sys.modules['DocumentWorking'])

doc = Document("/home/blitt/Academic/PSC/NLPPipelines/AG2PIAnalytics/AG2PISlides.csv",nlp,subsetColumns=["Persona", "Touchpoint", ["Touchpoint", "Persona"]])
personaDict = doc.getDoubleColSubAttributeDict(["Touchpoint", "Persona"], "workingDoc")

Boring emails that get buried in lost. Contacting faculty that are involved in their institution. Happy to discover that there is a group focusing on what she is looking for. Time conflict no easily accessible link to more information if you cant make the meeting. Connections and contacts with other research faculty. New ideas. Website for AG2PI that is loaded with details. Listserve email about events taking place. Hope that someone somewhere has information and will Google it. Finding Likeminded researchers. Looking for funding. Expanding on their topics of interest. Answers to questions of How have others dealt with USDA. Talk to coworkers about it. Feel disappointed that she didnt know about it earlier. Feels great that there is a place with her kind of researchers getting together regularly. Looking forward to adding further interactions to her calendar. Assessment of if they actually found what they were looking for and positive or negative feelings associated with if they found 

In [34]:
spanList = getSpans(personaDict["How do they first discover the existence of AG2PI & what are they looking for?"]["Scientist Susan"])
print(extractNouns(spanList))

[-2.6432e-01  1.9472e-01 -7.4001e-01 -1.0659e-02 -3.9336e-01  6.2499e-01
  2.5623e-01 -8.6893e-01  1.4548e-01  1.9867e+00 -2.0354e-01  1.9516e-01
  2.5106e-01 -8.4998e-01 -3.5718e-01  1.6608e-01 -1.9463e-01  8.4832e-01
 -2.1302e-01 -7.5432e-02 -1.9087e-01  3.4951e-01 -1.3711e-01  1.1313e-01
  3.9638e-01  1.5668e-01 -4.8505e-01 -2.0341e-01  2.1858e-01 -3.7724e-01
 -4.0292e-02  3.5468e-01 -2.6293e-01 -1.7104e-02 -2.2797e-01  2.0573e-01
 -9.1331e-01 -2.5559e-02  1.2478e-02 -3.4336e-01 -6.3638e-01 -2.2280e-01
 -1.9321e-01 -2.2934e-01 -3.8478e-01  4.0481e-01  2.0311e-01  1.0919e-01
  3.4384e-01 -2.0668e-01 -4.4387e-01 -1.3450e-01  2.5788e-03 -7.4796e-01
 -1.0846e-01  2.3540e-01 -4.9964e-01 -3.0562e-01  1.9370e-01  2.7532e-01
  2.2083e-01  2.7774e-01 -1.7171e-01 -2.7419e-01  2.9267e-01  1.4068e-01
 -8.5017e-02 -3.9748e-02 -1.4420e-01  7.1085e-01  2.8785e-01  7.2469e-02
  8.3715e-01 -2.2836e-01  5.5542e-01 -1.6718e-01  5.4128e-01 -2.2454e-01
 -2.4190e-01  7.2170e-01  3.5543e-01 -4.2787e-01 -3

In [ ]:
"""
Subset list is one of the lists from subsetColumns given when initializing the Document object. 
It specifies first what information we will be taking a slice of to create the tree, and 
what information will be used to color the tree. 

cleaningLevel is the level of information that we want to take. Right now we should always use "workingDoc",
since we need actual spaCy documents to get spans and calculate span similarity. 

Key is the "slice" to take. 


For example, we could have: 
key = 'How do they first discover the existence of AG2PI & what are they looking for?'
clusterSentEmbeddings(["Touchpoint", "Persona"], "workingDoc", key) 

which would take answers to the question "How do they first discover the existence of AG2PI & what are they looking for"
colored by Persona, using text from the workingDoc text created during the automatic cleaning of the data
that occurs when creating Document. 
"""

def clusterSentEmbeddings(subsetList, cleaningLevel, inKey, ax=None): 
    #example 
    #personaDict = doc.getDoubleColSubAttributeDict(["Touchpoint", "Persona"], "workingDoc")
    #inKey = 'How do they first discover the existence of AG2PI & what are they looking for?'
    
    personaDict = doc.getDoubleColSubAttributeDict(["Touchpoint", "Persona"], "workingDoc")
    
    """
    recreate the nested dictionary but get the spans instead of having the working doc
    this dictionary has touchpoints as the outer key and personas as the inner, so we can easily split by persona 
    for a particular toucpoint 
    """
    spanDict = {}
    for outerKey in personaDict.keys():
        spanDict[outerKey] = {}
        for innerKey in personaDict[outerKey].keys(): 
            spanDict[outerKey][innerKey] = getSpans(personaDict[outerKey][innerKey]) 

    #we most likely also only want to get dictionaries for one value of the touchpoint 
    keyDictSpans = spanDict[inKey]

    #now add spaces so there are technically no double spans in keyDict
    #use temporary dictionary to hold items while changing them 
    inSpans = {}

    #need to do deepcopy here
    keyDict = {}

    for key in keyDictSpans.keys(): 
        keyDict[key] = [None for item in range(0, len(keyDictSpans[key]))]
        for index, span in enumerate(keyDictSpans[key]): 
            spaceSpan = str(span)

            #add as many spaces to this span as needed so that there are no doubles, tripes
            #etc.. in our span list when we go to create plot/assign colors 
            while spaceSpan in inSpans: 
                spaceSpan += " "

            inSpans[spaceSpan] = True 

            #replace non-spaced version of span with space span 
            keyDict[key][index] = spaceSpan

    #get colors, map to keys in dictionary created above
    colorListFull = ["#e41a1c", "#377eb8", "#4daf4a", "#984ea3", "#ff7f00", "#ffff33", "#a65628", "#f781bf", "#999999"]
    colorListFull = [item.upper() for item in colorListFull]
    colorKeyMapping = dict(zip(sorted(list(keyDict.keys())), colorListFull[:len(keyDict.keys())]))

    #make sure legend order matches with colors
    legendElements = []
    for index, key in enumerate(sorted(list(keyDict.keys()))):
        currColor = colorListFull[index]
        legendElements.insert(0, Patch(facecolor=currColor,label=key))

    #map each span to the color associated with its key in keyDict
    #this way we can easily grab the color for any span when plotting 
    colorSpanMapping = {
        str(span):colorKeyMapping[key]
        for key, val in keyDict.items()
        for span in val}

    #NOTE: we should be ok matching the spans to the strings because we sort the keys when we iterate 
    #and we maintain the order of items in the lists within keyDictSpans 
    #create list of all spans 
    spanList = []
    for k in sorted(list(keyDictSpans.keys())): 
        spanList += keyDictSpans[k]

    #create list of all string versions of spans (with spaces )
    labelList = []
    for k in sorted(list(keyDict.keys())): 
        labelList += keyDict[k]
    
    